In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import sys
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

ImportError: No module named pycocotools.coco

In [2]:
dataDir='..'
dataType='val2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)

In [3]:
# initialize COCO api for instance annotations
coco_instance=COCO(annFile)

loading annotations into memory...
Done (t=8.40s)
creating index...
index created!


In [4]:
# initialize COCO api for caption annotations
annFile = '%s/annotations/captions_%s.json'%(dataDir,dataType)
coco_caps=COCO(annFile)

loading annotations into memory...
Done (t=0.91s)
creating index...
index created!


In [14]:
# load word dictionnary
d = 50 #100 or 200
#vector_db = 'glove.twitter.27B'
vector_db = 'glove.6B'
PATH_VECTORS = "../glove/%s/%s.%dd.txt" % (vector_db,vector_db,d)

with open(PATH_VECTORS, 'r') as f:
    word_vectors = {}
    for line in f:
        vals = line.rstrip().split(' ')
        word_vectors[unicode(vals[0],"utf-8")] = map(float, vals[1:])

In [16]:
# example of glove vector from the dictionnary
print word_vectors[u"fire"]

[0.50905, -0.36805, 0.41275, -0.44637, -0.017932, -0.12425, -0.43753, 0.52016, -0.1449, -0.83957, -0.32243, -0.71141, -0.43867, -0.13184, -0.27728, 0.011158, 0.097582, 0.5546, -1.3955, -0.58442, 0.25271, 0.74644, -0.18067, -0.46553, -0.09358, -1.4951, 0.2663, 0.55315, 1.103, -0.06003, 3.0834, -0.57123, -0.91293, -0.52011, 0.057919, 0.8553, 0.75373, -1.2263, 0.60596, 0.76143, -0.02809, -0.077767, -0.10431, -0.50895, 0.92005, 0.21507, -0.29191, -0.03356, 0.056012, -0.43283]


In [11]:
# store the whole val2014 images
imgs_val2014 = coco_instance.loadImgs(coco_instance.getImgIds())
nbImages = len(imgs_val2014)
print("%d image names loaded as val2014" % nbImages) 

# loop over the 5k first images in the val database
imgs_train_sub = coco_instance.loadImgs(coco_instance.getImgIds()[:10000])
#imgsNames = [img['file_name'] for img in imgs]
#imgsIds = [img['id'] for img in imgs]
nbImages = len(imgs_train_sub)
print("%d image names loaded as train_sub" % nbImages)


# loop over the test database in the val database
imgs_test_sub = coco_instance.loadImgs(coco_instance.getImgIds()[10000:12000])
#imgsNames = [img['file_name'] for img in imgs]
#imgsIds = [img['id'] for img in imgs]
nbImages = len(imgs_test_sub)
print("%d image names loaded as test_sub" % nbImages)

40504 image names loaded as val2014
10000 image names loaded as train_sub
2000 image names loaded as test_sub


In [20]:
# generate word category statistics over the train and test databases
# and compare them over the full datatbase
# test check to see if the subset is representative of the whole val2014

catstat_val2014 = {}

catstat_train = {}
catstat_test = {}

count_val2014 = 0
for img in imgs_val2014:
    annIds = coco_instance.getAnnIds(imgIds=img['id'])
    anns = coco_instance.loadAnns(annIds)
    for ann in anns:
        count_val2014 += 1
        instance = coco_instance.loadCats(ann['category_id'])[0]['name']
        if instance in catstat_val2014.keys():
            catstat_val2014[instance] += 1
        else:
            catstat_val2014[instance] = 1
            
count_train_sub = 0
for img in imgs_train_sub:
    annIds = coco_instance.getAnnIds(imgIds=img['id'])
    anns = coco_instance.loadAnns(annIds)
    
    for ann in anns:
        count_train_sub += 1
        instance = coco_instance.loadCats(ann['category_id'])[0]['name']
        if instance in catstat_train.keys():
            catstat_train[instance] += 1
        else:
            catstat_train[instance] = 1

count_test_sub = 0
for img in imgs_test_sub:
    annIds = coco_instance.getAnnIds(imgIds=img['id'])
    anns = coco_instance.loadAnns(annIds)
    
    for ann in anns:
        count_test_sub += 1
        instance = coco_instance.loadCats(ann['category_id'])[0]['name']
        if instance in catstat_test.keys():
            catstat_test[instance] += 1
        else:
            catstat_test[instance] = 1

# normalized stats
n_keys_val2014 = len(imgs_val2014)
n_keys_train = len(imgs_train_sub)
n_keys_test = len(imgs_test_sub)

for key in catstat_val2014.keys():
    catstat_val2014[key] = float(catstat_val2014[key]) / count_val2014

for key in catstat_train.keys():
    catstat_train[key] = float(catstat_train[key]) / count_train_sub

for key in catstat_test.keys():
    catstat_test[key] = float(catstat_test[key]) / count_test_sub

print 'catstat_val2014'
print catstat_val2014
print np.sum(catstat_val2014.values())
print 'catstat_train'
print catstat_train
print np.sum(catstat_train.values())
print 'catstat_test'
print catstat_test
print np.sum(catstat_test.values())


catstat_val2014
{u'toilet': 0.005012419700214133, u'fire hydrant': 0.0022269807280513917, u'sports ball': 0.00759914346895075, u'bicycle': 0.008476231263383297, u'kite': 0.009771306209850106, u'laptop': 0.006119057815845825, u'potted plant': 0.010542184154175589, u'tennis racket': 0.005570877944325482, u'teddy bear': 0.005283083511777302, u'donut': 0.008702355460385438, u'snowboard': 0.0027203426124197, u'carrot': 0.009195717344753747, u'motorcycle': 0.010535331905781585, u'oven': 0.004025695931477516, u'keyboard': 0.0035220556745182013, u'scissors': 0.0015211991434689507, u'chair': 0.045002141327623126, u'couch': 0.006602141327623126, u'mouse': 0.002915631691648822, u'airplane': 0.004950749464668094, u'boat': 0.012330620985010707, u'apple': 0.006105353319057816, u'sheep': 0.011018415417558887, u'horse': 0.007516916488222698, u'sandwich': 0.005005567451820129, u'banana': 0.010021413276231263, u'cup': 0.024106209850107068, u'tv': 0.007047537473233405, u'backpack': 0.00990492505353319, u

In [22]:
# generate category word vectors over images in the train database
words_train_sub = {}
counter = 0

for img in imgs_train_sub:
    counter += 1
    print '\rtrain_database_word_vector_gen:%d/%d' % (counter,len(imgs_train_sub)),
    sys.stdout.flush()
    
    annIds = coco_instance.getAnnIds(imgIds=img['id'])
    anns = coco_instance.loadAnns(annIds)
    den = 0
    vector = np.zeros(len(word_vectors[word_vectors.keys()[0]]))
    
    for ann in anns:
        instance = coco_instance.loadCats(ann['category_id'])[0]['name']
        instance_words = instance.split()        

        n_words_in_ann = 0
        vect_tmp = np.zeros(vector.shape)

        for word in instance_words:
            if word in word_vectors.keys():
                n_words_in_ann += 1
                vect_tmp = vect_tmp + np.array(word_vectors[word])        
            else:
                print '\nWORD NOT FOUND IN DICTIONNARY: %s' % word         

        vect_tmp = vect_tmp / n_words_in_ann
             
        den += ann['area']
        vector = vector + ann['area']*vect_tmp
        #if instance in word_vectors.keys(): # an instance can have a various number of words like "hot dog", "tennis racket"
        #   den += ann['area']
        #  vector = vector + ann['area']*np.array(word_vectors[instance])
        #else:
        #   print '\nWORD NOT FOUND IN DICTIONARY: %s' % instance
    
    if den==0:
        words_train_sub[img['id']] = np.zeros(vector.shape)
    else:
        words_train_sub[img['id']] = vector / den
    if counter==10:
        break

# Save image features
print("\nSaving features")
np.save('words_train_sub', words_train_sub) # save as .npy

# to retrieve vector of image with id=10878
# data = np.load('words_train_sub')
# data[()][10878] gets the corresponding vectors

train_database_word_vector_gen:10/5000         Saving features


In [ ]:
# generate category word vectors over images in the test database
words_test_sub = {}
counter = 0

for img in imgs_test_sub:
    counter += 1
    print '\rtest_database_word_vector_gen:%d/%d' % (counter,len(imgs_test_sub)),
    sys.stdout.flush()

    annIds = coco_instance.getAnnIds(imgIds=img['id'])
    anns = coco_instance.loadAnns(annIds)
    den = 0
    vector = np.zeros(len(word_vectors[word_vectors.keys()[0]]))

    for ann in anns:
        instance = coco_instance.loadCats(ann['category_id'])[0]['name']
        instance_words = instance.split()

        n_words_in_ann = 0
        vect_tmp = np.zeros(vector.shape)

        for word in instance_words:
            if word in word_vectors.keys():
                n_words_in_ann += 1
                vect_tmp = vect_tmp + np.array(word_vectors[word])
            else:
                print '\nWORD NOT FOUND IN DICTIONNARY: %s' % word

        vect_tmp = vect_tmp / n_words_in_ann

        den += ann['area']
        vector = vector + ann['area']*vect_tmp
        #if instance in word_vectors.keys(): # an instance can have a various number of words like "hot dog", "tennis racket"
        #   den += ann['area']
        #  vector = vector + ann['area']*np.array(word_vectors[instance])
        #else:
        #   print '\nWORD NOT FOUND IN DICTIONARY: %s' % instance

    if den==0:
        words_train_sub[img['id']] = np.zeros(vector.shape)
    else:
        words_train_sub[img['id']] = vector / den

# Save image features
print("\nSaving features")
np.save('words_test_sub', words_test_sub) # save as .npy